In [1]:
'''
#Sequence to sequence example in Keras (character-level).
This script demonstrates how to implement a basic character-level
sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.
**Summary of the algorithm**
- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
**Data download**
[English to French sentence pairs.
](http://www.manythings.org/anki/fra-eng.zip)
[Lots of neat sentence pairs datasets.
](http://www.manythings.org/anki/)
**References**
- [Sequence to Sequence Learning with Neural Networks
   ](https://arxiv.org/abs/1409.3215)
- [Learning Phrase Representations using
    RNN Encoder-Decoder for Statistical Machine Translation
    ](https://arxiv.org/abs/1406.1078)
''';

In [2]:
from __future__ import print_function

In [4]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

In [1]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 15000  # Number of samples to train on.
# Path to the data txt file on disk.
data_path = 'fra-eng/fra.txt'

In [2]:
# Vectorize the data.
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, 'r') as f: #, encoding='utf-8'
    lines = f.read().split('\n')

In [3]:
lines

['Go.\tVa !',
 'Hi.\tSalut !',
 'Run!\tCours\u202f!',
 'Run!\tCourez\u202f!',
 'Who?\tQui ?',
 'Wow!\tÇa alors\u202f!',
 'Fire!\tAu feu !',
 "Help!\tÀ l'aide\u202f!",
 'Jump.\tSaute.',
 'Stop!\tÇa suffit\u202f!',
 'Stop!\tStop\u202f!',
 'Stop!\tArrête-toi !',
 'Wait!\tAttends !',
 'Wait!\tAttendez !',
 'Go on.\tPoursuis.',
 'Go on.\tContinuez.',
 'Go on.\tPoursuivez.',
 'Hello!\tBonjour !',
 'Hello!\tSalut !',
 'I see.\tJe comprends.',
 "I try.\tJ'essaye.",
 "I won!\tJ'ai gagné !",
 "I won!\tJe l'ai emporté !",
 'I won.\tJ’ai gagné.',
 'Oh no!\tOh non !',
 'Attack!\tAttaque !',
 'Attack!\tAttaquez !',
 'Cheers!\tSanté !',
 'Cheers!\tÀ votre santé !',
 'Cheers!\tMerci !',
 'Cheers!\tTchin-tchin !',
 'Get up.\tLève-toi.',
 'Go now.\tVa, maintenant.',
 'Go now.\tAllez-y maintenant.',
 'Go now.\tVas-y maintenant.',
 "Got it!\tJ'ai pigé !",
 'Got it!\tCompris !',
 'Got it?\tPigé\u202f?',
 'Got it?\tCompris\u202f?',
 "Got it?\tT'as capté\u202f?",
 'Hop in.\tMonte.',
 'Hop in.\tMontez.',
 'Hu

In [4]:
print(f"We are only going to train with {num_samples:,.0f} samples")
print(f'However, we have {len(lines) - 1:,.0f} potental french-english pairs to work with')

We are only going to train with 15,000 samples
However, we have 167,130 potental french-english pairs to work with


In [5]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [6]:
len(input_texts)

15000

In [7]:
input_texts[9999]

"Don't resist us."

In [8]:
target_texts[::-1]

['\tTon chien est-il méchant\u202f?\n',
 '\tVotre voiture est-elle rapide ?\n',
 "\tC'est votre stylo\u202f?\n",
 "\tC'est ton stylo\u202f?\n",
 '\tEst-ce votre chien ?\n',
 '\tEst-ce ton chien ?\n',
 '\tEst-ce votre voiture\u202f?\n',
 '\tEst-ce ta voiture\u202f?\n',
 "\tEst-ce que c'est votre voiture\u202f?\n",
 '\tEst-ce ton DVD\u202f?\n',
 "\tEst-ce que c'est trop haut\xa0?\n",
 "\tC'est à vendre\xa0?\n",
 '\tEst-ce que ceci est à vendre\xa0?\n',
 '\tEst-ce une énigme ?\n',
 "\tS'agit-il d'une lettre ?\n",
 '\tSuis-je bien au 223-1374 ?\n',
 '\tÀ quoi ça sert ?\n',
 '\tÀ quoi cela sert-il ?\n',
 '\tLa porte est-elle ouverte ?\n',
 '\tLa banque est-elle ouverte\u202f?\n',
 '\tEst-ce que la banque est ouverte\u202f?\n',
 '\tEst-ce votre mère\xa0?\n',
 '\tEst-ce là ton chien ?\n',
 '\tEst-ce là votre chien ?\n',
 '\tEst-ce votre chatte ?\n',
 '\tEst-ce ta chatte ?\n',
 '\tEst-ce votre chat ?\n',
 '\tEst-ce ton chat\xa0?\n',
 '\tEst-ce là ta voiture\u202f?\n',
 '\tEst-ce ta voiture\u20

In [9]:
print(target_characters)

{'ê', '.', 'l', '‘', '-', 'H', '5', '2', 'P', '0', 'A', 'x', '&', 'z', '\u2009', 'M', 'Ê', 'Ô', 'U', 'â', 'b', 'K', 'É', 'ô', 'î', '\xa0', 'V', '»', 'w', 'm', '!', 'o', 'Z', 'j', 'I', '3', 'ë', 'u', 'a', '1', ' ', 'n', 'à', 'F', '7', '8', 'p', 'v', 'Y', 'O', 'e', 'T', 'Ç', 'û', 'r', 'i', 'À', 'q', '«', 'J', 'Q', 'y', "'", 'f', 'G', 't', '\n', 'é', '’', 'è', 'B', '\t', 'N', 'h', 'C', 'E', ')', 's', 'L', ',', 'R', 'ù', 'k', '4', 'c', '(', 'ï', 'd', 'ç', '6', '?', ':', '$', 'D', '9', '\u202f', 'g', '%', 'S', 'œ'}


In [10]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])
print(f"There are {num_encoder_tokens} unique encoder tokens")
print(f"There are {num_decoder_tokens} unique decoder tokens")
print(f"The max encoder seq length is {max_encoder_seq_length}")
print(f"The max decoder seq length is {max_decoder_seq_length}")

There are 73 unique encoder tokens
There are 100 unique decoder tokens
The max encoder seq length is 17
The max decoder seq length is 59


In [11]:
print('Number of samples:', len(input_texts))
print('Number of targets:', len(target_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 15000
Number of targets: 15000
Number of unique input tokens: 73
Number of unique output tokens: 100
Max sequence length for inputs: 17
Max sequence length for outputs: 59


In [236]:
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

In [237]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [238]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 "'": 4,
 ',': 5,
 '-': 6,
 '.': 7,
 '0': 8,
 '1': 9,
 '3': 10,
 '8': 11,
 '9': 12,
 '?': 13,
 'A': 14,
 'B': 15,
 'C': 16,
 'D': 17,
 'E': 18,
 'F': 19,
 'G': 20,
 'H': 21,
 'I': 22,
 'J': 23,
 'K': 24,
 'L': 25,
 'M': 26,
 'N': 27,
 'O': 28,
 'P': 29,
 'Q': 30,
 'R': 31,
 'S': 32,
 'T': 33,
 'U': 34,
 'V': 35,
 'a': 36,
 'b': 37,
 'c': 38,
 'd': 39,
 'e': 40,
 'f': 41,
 'g': 42,
 'h': 43,
 'i': 44,
 'j': 45,
 'k': 46,
 'l': 47,
 'm': 48,
 'n': 49,
 'o': 50,
 'p': 51,
 'q': 52,
 'r': 53,
 's': 54,
 't': 55,
 'u': 56,
 'v': 57,
 'x': 58,
 'y': 59,
 'z': 60,
 '\xa0': 61,
 'À': 62,
 'Ç': 63,
 'É': 64,
 'à': 65,
 'â': 66,
 'ç': 67,
 'è': 68,
 'é': 69,
 'ê': 70,
 'î': 71,
 'ô': 72,
 'û': 73,
 '\u2009': 74,
 '’': 75,
 '\u202f': 76}

In [239]:
target_texts[0]

'\tVa !\n'

In [240]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        print(char)
        print(target_token_index[char])
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

	
0
V
35
a
36
 
2
!
3


1
	
0
S
32
a
36
l
47
u
56
t
55
 
2
!
3


1
	
0
C
16
o
50
u
56
r
53
s
54
 
76
!
3


1
	
0
C
16
o
50
u
56
r
53
e
40
z
60
 
76
!
3


1
	
0
Q
30
u
56
i
44
 
2
?
13


1
	
0
Ç
63
a
36
 
2
a
36
l
47
o
50
r
53
s
54
 
76
!
3


1
	
0
A
14
u
56
 
2
f
41
e
40
u
56
 
2
!
3


1
	
0
À
62
 
2
l
47
'
4
a
36
i
44
d
39
e
40
 
76
!
3


1
	
0
S
32
a
36
u
56
t
55
e
40
.
7


1
	
0
Ç
63
a
36
 
2
s
54
u
56
f
41
f
41
i
44
t
55
 
76
!
3


1
	
0
S
32
t
55
o
50
p
51
 
76
!
3


1
	
0
A
14
r
53
r
53
ê
70
t
55
e
40
-
6
t
55
o
50
i
44
 
2
!
3


1
	
0
A
14
t
55
t
55
e
40
n
49
d
39
s
54
 
2
!
3


1
	
0
A
14
t
55
t
55
e
40
n
49
d
39
e
40
z
60
 
2
!
3


1
	
0
P
29
o
50
u
56
r
53
s
54
u
56
i
44
s
54
.
7


1
	
0
C
16
o
50
n
49
t
55
i
44
n
49
u
56
e
40
z
60
.
7


1
	
0
P
29
o
50
u
56
r
53
s
54
u
56
i
44
v
57
e
40
z
60
.
7


1
	
0
B
15
o
50
n
49
j
45
o
50
u
56
r
53
 
2
!
3


1
	
0
S
32
a
36
l
47
u
56
t
55
 
2
!
3


1
	
0
J
23
e
40
 
2
c
38
o
50
m
48
p
51
r
53
e
40
n
49
d
39
s
54
.
7


1
	
0
J
23
'
4
e


40
 
2
c
38
h
43
e
40
z
60
 
2
t
55
o
50
i
44
.
7


1
	
0
R
31
e
40
n
49
t
55
r
53
e
40
z
60
 
2
c
38
h
43
e
40
z
60
 
2
v
57
o
50
u
56
s
54
.
7


1
	
0
V
35
a
36
 
2
d
39
o
50
u
56
c
38
e
40
m
48
e
40
n
49
t
55
 
2
!
3


1
	
0
A
14
l
47
l
47
e
40
z
60
 
2
d
39
o
50
u
56
c
38
e
40
m
48
e
40
n
49
t
55
 
2
!
3


1
	
0
À
62
 
2
l
47
a
36
 
2
r
53
e
40
v
57
o
50
y
59
u
56
r
53
e
40
.
7


1
	
0
A
14
t
55
t
55
e
40
n
49
d
39
s
54
 
2
u
56
n
49
 
2
p
51
e
40
u
56
 
2
!
3


1
	
0
A
14
t
55
t
55
e
40
n
49
d
39
e
40
z
60
 
2
u
56
n
49
 
2
p
51
e
40
u
56
 
2
!
3


1
	
0
T
33
i
44
e
40
n
49
s
54
 
2
b
37
o
50
n
49
 
2
!
3


1
	
0
T
33
e
40
n
49
e
40
z
60
 
2
b
37
o
50
n
49
 
2
!
3


1
	
0
I
22
l
47
 
2
l
47
a
36
i
44
s
54
s
54
a
36
 
2
t
55
o
50
m
48
b
37
e
40
r
53
.
7


1
	
0
I
22
l
47
 
2
a
36
 
2
l
47
a
36
i
44
s
54
s
54
é
69
 
2
t
55
o
50
m
48
b
37
e
40
r
53
.
7


1
	
0
I
22
l
47
 
2
c
38
o
50
u
56
r
53
t
55
.
7


1
	
0
A
14
i
44
d
39
e
40
-
6
m
48
o
50
i
44
 
2
!
3


1
	
0
A
14
i
44
d
39
e
40

e
40
n
49
.
7


1
	
0
A
14
p
51
p
51
e
40
l
47
l
47
e
40
 
2
T
33
o
50
m
48
.
7


1
	
0
A
14
p
51
p
51
e
40
l
47
e
40
z
60
 
2
T
33
o
50
m
48
.
7


1
	
0
C
16
o
50
u
56
r
53
a
36
g
42
e
40
 
76
!
3


1
	
0
D
17
é
69
t
55
e
40
n
49
d
39
s
54
-
6
t
55
o
50
i
44
 
76
!
3


1
	
0
M
26
e
40
n
49
o
50
t
55
t
55
e
40
z
60
-
6
l
47
e
40
.
7


1
	
0
A
14
v
57
a
36
n
49
c
38
e
40
 
2
!
3


1
	
0
A
14
v
57
a
36
n
49
c
38
e
40
z
60
 
2
!
3


1
	
0
C
16
o
50
n
49
t
55
i
44
n
49
u
56
e
40
 
2
à
65
 
2
r
53
o
50
u
56
l
47
e
40
r
53
 
2
!
3


1
	
0
C
16
o
50
n
49
t
55
i
44
n
49
u
56
e
40
z
60
 
2
à
65
 
2
r
53
o
50
u
56
l
47
e
40
r
53
 
2
!
3


1
	
0
T
33
r
53
o
50
u
56
v
57
e
40
 
2
T
33
o
50
m
48
.
7


1
	
0
T
33
r
53
o
50
u
56
v
57
e
40
z
60
 
2
T
33
o
50
m
48
.
7


1
	
0
R
31
é
69
p
51
a
36
r
53
e
40
z
60
 
2
c
38
e
40
c
38
i
44
.
7


1
	
0
R
31
é
69
p
51
a
36
r
53
e
40
 
2
ç
67
a
36
.
7


1
	
0
L
25
â
66
c
38
h
43
e
40
-
6
t
55
o
50
i
44
 
2
!
3


1
	
0
D
17
e
40
s
54
c
38
e
40
n
49
d
39
s
54
 
2

2
c
38
o
50
n
49
t
55
e
40
n
49
t
55
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
c
38
h
43
e
40
z
60
 
2
m
48
o
50
i
44
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
e
40
n
49
 
2
r
53
e
40
t
55
a
36
r
53
d
39
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
p
51
a
36
r
53
e
40
s
54
s
54
e
40
u
56
x
58
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
f
41
a
36
i
44
n
49
é
69
a
36
n
49
t
55
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
p
51
a
36
r
53
e
40
s
54
s
54
e
40
u
56
s
54
e
40
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
f
41
a
36
i
44
n
49
é
69
a
36
n
49
t
55
e
40
.
7


1
	
0
J
23
e
40
 
2
v
57
a
36
i
44
s
54
 
2
b
37
i
44
e
40
n
49
.
7


1
	
0
J
23
e
40
 
2
m
48
e
40
 
2
p
51
o
50
r
53
t
55
e
40
 
2
b
37
i
44
e
40
n
49
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
r
53
i
44
c
38
h
43
e
40
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
e
40
n
49
 
2
s
54
é
69
c
38
u
56
r
53
i
44
t
55
é
69
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
m
48
a
36
l


h
43
e
40
z
60
 
2
m
48
o
50
i
44
 
2
!
3


1
	
0
V
35
i
44
e
40
n
49
s
54
 
2
b
37
i
44
e
40
n
49
t
55
ô
72
t
55
 
2
!
3


1
	
0
V
35
e
40
n
49
e
40
z
60
 
2
b
37
i
44
e
40
n
49
t
55
ô
72
t
55
 
2
!
3


1
	
0
C
16
a
36
l
47
m
48
e
40
z
60
-
6
v
57
o
50
u
56
s
54
 
2
!
3


1
	
0
A
14
i
44
-
6
j
45
e
40
 
2
g
42
a
36
g
42
n
49
é
69
 
2
?
13


1
	
0
L
25
'
4
a
36
i
44
-
6
j
45
e
40
 
2
e
40
m
48
p
51
o
50
r
53
t
55
é
69
 
2
?
13


1
	
0
E
18
s
54
t
55
-
6
c
38
e
40
 
2
m
48
o
50
i
44
 
2
q
52
u
56
i
44
 
2
a
36
i
44
 
2
g
42
a
36
g
42
n
49
é
69
 
2
?
13


1
	
0
F
19
a
36
i
44
t
55
e
40
s
54
-
6
l
47
e
40
 
2
m
48
a
36
i
44
n
49
t
55
e
40
n
49
a
36
n
49
t
55
.
7


1
	
0
D
17
e
40
s
54
 
2
c
38
h
43
i
44
e
40
n
49
s
54
 
2
a
36
b
37
o
50
i
44
e
40
n
49
t
55
.
7


1
	
0
L
25
e
40
s
54
 
2
c
38
h
43
i
44
e
40
n
49
s
54
 
2
a
36
b
37
o
50
i
44
e
40
n
49
t
55
.
7


1
	
0
N
27
e
40
 
2
d
39
e
40
m
48
a
36
n
49
d
39
e
40
 
2
p
51
a
36
s
54
 
2
!
3


1
	
0
N
27
e
40
 
2
p
51
l
47
e
40
u
56
r
53
e

a
36
i
44
m
48
e
40
 
2
ç
67
a
36
.
7


1
	
0
J
23
e
40
 
2
l
47
’
75
a
36
i
44
 
2
p
51
e
40
r
53
d
39
u
56
.
7


1
	
0
J
23
'
4
a
36
d
39
o
50
r
53
e
40
 
2
ç
67
a
36
 
2
!
3


1
	
0
J
23
'
4
a
36
d
39
o
50
r
53
e
40
 
2
ç
67
a
36
 
2
!
3


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
s
54
é
69
r
53
i
44
e
40
u
56
x
58
 
76
!
3


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
s
54
é
69
r
53
i
44
e
40
u
56
x
58
.
7


1
	
0
J
23
e
40
 
2
d
39
o
50
i
44
s
54
 
2
y
59
 
2
a
36
l
47
l
47
e
40
r
53
.
7


1
	
0
I
22
l
47
 
2
f
41
a
36
u
56
t
55
 
2
q
52
u
56
e
40
 
2
j
45
'
4
y
59
 
2
a
36
i
44
l
47
l
47
e
40
.
7


1
	
0
I
22
l
47
 
2
m
48
e
40
 
2
f
41
a
36
u
56
t
55
 
2
y
59
 
2
a
36
l
47
l
47
e
40
r
53
.
7


1
	
0
I
22
l
47
 
2
m
48
e
40
 
2
f
41
a
36
u
56
t
55
 
2
p
51
a
36
r
53
t
55
i
44
r
53
.
7


1
	
0
I
22
l
47
 
2
m
48
e
40
 
2
f
41
a
36
u
56
t
55
 
2
m
48
'
4
e
40
n
49
 
2
a
36
l
47
l
47
e
40
r
53
.
7


1
	
0
J
23
e
40
 
2
d
39
o
50
i
44
s
54
 
2
p
51
a
36
r
53
t
55
i
44
r
53
.
7


1


e
40
 
2
s
54
u
56
i
44
s
54
 
2
c
38
h
43
a
36
n
49
c
38
e
40
u
56
x
58
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
c
38
h
43
a
36
n
49
c
38
e
40
u
56
s
54
e
40
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
e
40
n
49
 
2
t
55
r
53
a
36
i
44
n
49
 
2
d
39
e
40
 
2
m
48
e
40
n
49
t
55
i
44
r
53
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
n
49
u
56
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
n
49
u
56
e
40
.
7


1
	
0
J
23
e
40
 
2
m
48
e
40
 
2
t
55
r
53
o
50
u
56
v
57
e
40
 
2
n
49
u
56
.
7


1
	
0
J
23
e
40
 
2
m
48
e
40
 
2
t
55
r
53
o
50
u
56
v
57
e
40
 
2
n
49
u
56
e
40
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
à
65
 
2
p
51
o
50
i
44
l
47
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
t
55
r
53
a
36
n
49
q
52
u
56
i
44
l
47
l
47
e
40
.
7


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
p
51
r
53
ê
70
t
55
e
40
 
2
!
3


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
p
51
r
53
ê
70
t
55
 
2
!
3


1
	
0
J
23
e
40
 
2
s
54
u
56
i
44
s
54
 
2
p
51
r
53
ê
70


42
a
36
r
53
d
39
e
40
 
2
d
39
e
40
r
53
r
53
i
44
è
68
r
53
e
40
 
76
!
3


1
	
0
R
31
e
40
g
42
a
36
r
53
d
39
e
40
 
2
i
44
c
38
i
44
.
7


1
	
0
R
31
e
40
g
42
a
36
r
53
d
39
e
40
z
60
 
2
i
44
c
38
i
44
.
7


1
	
0
É
64
c
38
h
43
a
36
u
56
f
41
f
41
e
40
-
6
t
55
o
50
i
44
 
2
!
3


1
	
0
É
64
c
38
h
43
a
36
u
56
f
41
f
41
e
40
z
60
-
6
v
57
o
50
u
56
s
54
 
2
!
3


1
	
0
D
17
é
69
t
55
e
40
n
49
d
39
s
54
-
6
t
55
o
50
i
44
 
2
!
3


1
	
0
L
25
a
36
i
44
s
54
s
54
e
40
-
6
t
55
o
50
i
44
 
2
a
36
l
47
l
47
e
40
r
53
 
2
!
3


1
	
0
L
25
a
36
i
44
s
54
s
54
e
40
z
60
-
6
v
57
o
50
u
56
s
54
 
2
a
36
l
47
l
47
e
40
r
53
 
2
!
3


1
	
0
B
15
o
50
u
56
g
42
e
40
 
2
d
39
e
40
 
2
l
47
à
65
.
7


1
	
0
P
29
o
50
u
56
s
54
s
54
e
40
z
60
-
6
v
57
o
50
u
56
s
54
.
7


1
	
0
P
29
o
50
u
56
s
54
s
54
e
40
-
6
t
55
o
50
i
44
.
7


1
	
0
J
23
o
50
l
47
i
44
 
2
c
38
o
50
u
56
p
51
 
2
!
3


1
	
0
P
29
o
50
u
56
r
53
 
2
s
54
û
73
r
53
.
7


1
	
0
M
26
a
36
i
44
s
54
 
2
o
50
u
56
a
36
i
44



1
	
0
A
14
t
55
t
55
e
40
n
49
t
55
i
44
o
50
n
49
 
2
!
3


1
	
0
F
19
a
36
i
44
t
55
e
40
s
54
 
2
a
36
t
55
t
55
e
40
n
49
t
55
i
44
o
50
n
49
 
76
!
3


1
	
0
F
19
a
36
i
44
s
54
 
2
a
36
t
55
t
55
e
40
n
49
t
55
i
44
o
50
n
49
 
76
!
3


1
	
0
N
27
o
50
u
56
s
54
 
2
s
54
o
50
m
48
m
48
e
40
s
54
 
2
t
55
o
50
m
48
b
37
é
69
s
54
 
2
d
39
'
4
a
36
c
38
c
38
o
50
r
53
d
39
.
7


1
	
0
N
27
o
50
u
56
s
54
 
2
a
36
v
57
o
50
n
49
s
54
 
2
r
53
é
69
u
56
s
54
s
54
i
44
 
76
!
3


1
	
0
N
27
o
50
u
56
s
54
 
2
a
36
v
57
o
50
n
49
s
54
 
2
r
53
é
69
u
56
s
54
s
54
i
44
 
76
!
3


1
	
0
N
27
o
50
u
56
s
54
 
2
l
47
'
4
a
36
v
57
o
50
n
49
s
54
 
2
f
41
a
36
i
44
t
55
.
7


1
	
0
N
27
o
50
u
56
s
54
 
2
a
36
v
57
o
50
n
49
s
54
 
2
o
50
u
56
b
37
l
47
i
44
é
69
.
7


1
	
0
N
27
o
50
u
56
s
54
 
2
l
47
'
4
a
36
v
57
o
50
n
49
s
54
 
2
v
57
u
56
.
7


1
	
0
N
27
o
50
u
56
s
54
 
2
l
47
'
4
a
36
v
57
o
50
n
49
s
54
 
2
v
57
u
56
e
40
.
7


1
	
0
N
27
o
50
u
56
s
54
 
2
a
36
v
57
o
50
n
49


In [241]:
decoder_input_data[0, 4]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [242]:
decoder_target_data[0,4]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [243]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [244]:
encoder_outputs

<tf.Tensor 'lstm_3/TensorArrayReadV3:0' shape=(?, 256) dtype=float32>

In [245]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [246]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [253]:
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
# Save model
model.save('s2s.h5')

Train on 800 samples, validate on 200 samples
Epoch 1/100
800/800 [==============================] - 5s 6ms/step - loss: 0.0641 - acc: 0.4273 - val_loss: 1.6098 - val_acc: 0.2305
Epoch 2/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0411 - acc: 0.4328 - val_loss: 1.6245 - val_acc: 0.2250
Epoch 3/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0431 - acc: 0.4325 - val_loss: 1.6050 - val_acc: 0.2302
Epoch 4/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0430 - acc: 0.4320 - val_loss: 1.5923 - val_acc: 0.2327
Epoch 5/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0423 - acc: 0.4325 - val_loss: 1.6408 - val_acc: 0.2285
Epoch 6/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0422 - acc: 0.4324 - val_loss: 1.6421 - val_acc: 0.2286
Epoch 7/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0416 - acc: 0.4334 - val_loss: 1.6399 - val_acc: 0.2338
Epoch 8/100
800/800 [==

Epoch 62/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0347 - acc: 0.4330 - val_loss: 1.7151 - val_acc: 0.2303
Epoch 63/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0370 - acc: 0.4330 - val_loss: 1.7154 - val_acc: 0.2298
Epoch 64/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0347 - acc: 0.4333 - val_loss: 1.7168 - val_acc: 0.2305
Epoch 65/100
800/800 [==============================] - 3s 3ms/step - loss: 0.0354 - acc: 0.4330 - val_loss: 1.7382 - val_acc: 0.2276
Epoch 66/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0362 - acc: 0.4328 - val_loss: 1.7247 - val_acc: 0.2306
Epoch 67/100
800/800 [==============================] - 2s 3ms/step - loss: 0.0356 - acc: 0.4339 - val_loss: 1.7164 - val_acc: 0.2303
Epoch 68/100
800/800 [==============================] - 3s 3ms/step - loss: 0.0358 - acc: 0.4333 - val_loss: 1.7278 - val_acc: 0.2279
Epoch 69/100
800/800 [==============================] - 3s 4ms

/anaconda3/lib/python3.7/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [249]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

In [250]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

In [251]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        #print(output_tokens)
        #print(output_tokens[0, 0, 2])
        print(states_value[1].shape)
        print(states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [252]:
for seq_index in range(2):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    #print(input_seq)
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

(1, 256)
[array([[ 1.74049690e-01,  1.01413287e-01, -2.18195811e-01,
        -1.88570559e-01,  6.59997165e-01, -3.07717323e-01,
        -2.88076252e-01, -3.93630296e-01,  5.40675879e-01,
         2.79174984e-01, -3.75758588e-01,  1.83972027e-02,
         9.74219963e-02,  1.43724501e-01, -2.05511585e-01,
        -3.83587070e-02,  7.74086826e-03,  1.17249794e-01,
         1.45910800e-01, -7.28498995e-02, -3.80688995e-01,
        -4.28968579e-01,  2.72060275e-01, -1.56482100e-01,
         6.06665611e-01, -5.54875322e-02, -3.69019993e-02,
        -3.65527272e-01, -1.24136962e-01, -2.11800590e-01,
         2.53589332e-01,  3.37730497e-01, -3.33062977e-01,
         8.17978919e-01,  2.76785284e-01,  2.50910968e-01,
         3.58735889e-01,  1.93188176e-01,  9.14227605e-01,
         1.01387035e-02, -7.17083737e-02, -3.30238491e-01,
         1.20196491e-01, -3.07589829e-01,  7.15175986e-01,
        -9.05699730e-02, -1.49270609e-01, -1.46433517e-01,
         5.54081142e-01, -6.40270710e-02, -8.5

In [165]:
encoder_input_data[2].shape

(8, 43)

In [140]:
max_encoder_seq_length

8

In [146]:
def encode_sentence_english(sentence):
    encoder_input_sent = np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
    
    for t, char in enumerate(sentence):
        print('time', t)
        print('char', char)
        encoder_input_sent[0, t, input_token_index[char]] = 1
        
    return encoder_input_sent

In [260]:
input_sentence = "Attack!"

input_sentence_oneHot = encode_sentence_english(input_sentence)

decoded_sentence = decode_sequence(input_sentence_oneHot)

print(input_sentence)
print(decoded_sentence)

time 0
char A
time 1
char t
time 2
char t
time 3
char a
time 4
char c
time 5
char k
time 6
char !
(1, 256)
[array([[ 6.67638242e-01,  4.16984618e-01, -6.60817862e-01,
         7.32164681e-01, -7.08946511e-02, -2.96950042e-01,
        -4.30774093e-01, -0.00000000e+00,  5.78012824e-01,
        -0.00000000e+00,  0.00000000e+00,  9.81736124e-01,
         0.00000000e+00,  3.15692299e-03,  1.00755185e-01,
        -4.27727791e-04, -0.00000000e+00, -2.34443247e-02,
         5.63348770e-01,  4.70298499e-01, -8.85704041e-01,
        -4.11666811e-01,  5.36973894e-01, -0.00000000e+00,
         0.00000000e+00, -1.93124323e-03, -0.00000000e+00,
         8.56291652e-01, -7.59783745e-01,  3.83032556e-03,
        -1.57281775e-02,  3.62096012e-01, -0.00000000e+00,
         7.74557292e-01,  8.35497305e-02, -9.72686410e-01,
        -0.00000000e+00,  5.09300888e-01,  9.41330910e-01,
         0.00000000e+00, -1.52162090e-02, -2.32992426e-01,
        -9.98246610e-01,  3.08455020e-01, -6.15420640e-01,
       